# Resampling for expectation value calculations

**Pedro Rivero**  
Technical Lead @ IBM Quantum  
pedro.rivero@ibm.com

<!-- - **Co-authors**: *Abdullah Ash Saki, Ritajit Majumdar, Haimeng Zhang*
- **Collaborators**: *Drew Vandeth, Mirko Amico, Nate Earnest-Noble, Borja Peropadre* -->

In [1]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(1)
circuit.h(0)
circuit.measure_all()

In [2]:
from qiskit.primitives import Sampler

SHOTS = 10_000

sampler = Sampler()
job = sampler.run(circuit, shots=SHOTS)
result = job.result()

## Theoretical calculation

In [3]:
# Observe-like functions may be produced directly from (all-commuting) SparsePauliOp inputs
def observe(readout):
    """Observe Pauli-Z."""
    return +1 if readout == 0 else -1

In [4]:
from numpy import sqrt

readout_dist = result.quasi_dists[0]
oberved_dist = {observe(readout): probability for readout, probability in readout_dist.items()}

average = sum(observation * probability for observation, probability in oberved_dist.items())
variance = sum(probability * (observation - average)**2 for observation, probability in oberved_dist.items())
variance *= SHOTS / (SHOTS - 1)  # Note: variance adjustment to remove bias
std_error = sqrt(variance / SHOTS)  # Note: std error of the mean, not of the samples

print("Mean:", average)  # ≈ 0.0
# print("Variance:", variance)  # ≈ 1.0
print("Std. error:", std_error)  # ≈ variance / sqrt(SHOTS)

Mean: 0.0040000000000000036
Std. error: 0.010000420033182807


## Bootstrapping

In [5]:
def compute_expval(result, *args, **kwargs):
    """Compute expecation value from result."""
    readout_dist = result.quasi_dists[0]
    oberved_dist = {observe(readout): probability for readout, probability in readout_dist.items()}
    average = sum(observation * probability for observation, probability in oberved_dist.items())
    return average

In [6]:
from quantum_enablement.resampling import Bootstrap

NUM_RESAMPLES = 1_000

bootstrap = Bootstrap(seed=0)
resampled_results = [bootstrap.resample_result(result) for _ in range(NUM_RESAMPLES)]
bootstrap_expvals = [compute_expval(result) for result in resampled_results]

In [7]:
from numpy import mean, var, sqrt

average = mean(bootstrap_expvals)
variance = var(bootstrap_expvals)
variance *= NUM_RESAMPLES / (NUM_RESAMPLES - 1)  # Note: variance adjustment to remove bias
std_error = sqrt(variance)

print("Mean:", average)
print("Std. error:", std_error)

Mean: 0.004105000000000002
Std. error: 0.009809230142880529
